In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#import wrangle
import env
import seaborn as sns
import acquire
import prep

In [2]:
#single family residence from zil
df = acquire.wrangle_zillow()

In [3]:
df = prep.drop_columns(df)

In [4]:
df.columns

Index(['transactiondate', 'id', 'bathrooms', 'bedrooms', 'sqft', 'tax_value',
       'id', 'parcelid', 'bathroomcnt', 'bedroomcnt', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fips',
       'fullbathcnt', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'propertylandusetypeid',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'yearbuilt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'censustractandblock'],
      dtype='object')

In [5]:
df.head().T

,0,1,2,3,4
transactiondate,2017-01-02,2017-01-02,2017-01-02,2017-01-03,2017-01-03
id,20,31,33,110,111
bathrooms,2,4,2,1,2
bedrooms,4,3,3,3,3
sqft,3633,1620,2077,1244,1300
tax_value,296425,847770,646760,169471,233266
id,20,31,33,110,111
parcelid,11324547,11544747,11585547,12716947,12757147
bathroomcnt,2,4,2,1,2
bedroomcnt,4,3,3,3,3


In [7]:
df.isnull().sum().sort_values(ascending=False)

regionidcity                    1008
lotsizesquarefeet                300
censustractandblock              263
finishedsquarefeet12             186
taxamount                         71
regionidzip                       64
structuretaxvaluedollarcnt        62
yearbuilt                         27
calculatedbathnbr                 20
fullbathcnt                       20
bathroomcnt                        0
bedroomcnt                         0
id                                 0
bathrooms                          0
parcelid                           0
bedrooms                           0
calculatedfinishedsquarefeet       0
tax_value                          0
sqft                               0
id                                 0
latitude                           0
fips                               0
longitude                          0
propertycountylandusecode          0
propertylandusetypeid              0
rawcensustractandblock             0
regionidcounty                     0
r